In [35]:
steps8= """
time : 01-09-2024 03:00 
Steps : 0.0

time : 01-09-2024 04:00 
Steps : 10.0

time : 01-09-2024 05:00 
Steps : 20.0

time : 01-09-2024 06:00 
Steps : 124.0

time : 01-09-2024 07:00 
Steps : 25.0

time : 01-09-2024 08:00 
Steps : 10.0

time : 01-09-2024 09:00 
Steps : 10.0

time : 01-09-2024 10:00 
Steps : 5.0

time : 01-09-2024 11:00 
Steps : 50.0
"""

In [36]:
food8= """
time : 01-09-2024 08:30
foodName : Idli  (Protein- 4.35 g, Carbs- 29.05 g, Energy- 143.01 Kcal, Fiber- 1.89 g)

time : 01-09-2024 08:30
foodName : Vada (Protein- 4.84 g, Carbs- 10.7 g, Energy- 175.03 Kcal, Fiber- 2.5 g)

time : 01-09-2024 08:30
foodName : Sambar (Protein- 5.18 g, Carbs- 19.7 g, Energy- 140.82 Kcal, Fiber- 4.13 g)

time : 01-09-2024 10:30
foodName : Tea (Protein- 3.58 g, Carbs- 12.64 g, Energy- 101.45 Kcal, Fiber- 0.0 g)

time : 01-09-2024 10:30
foodName : Marie light biscuit (Protein- 0.94 g, Carbs- 7.4 g, Energy- 45.7 Kcal, Fiber- 0.0 g)
"""

In [37]:
cgm8="""
CGM readings starting from : 2024-09-01 03:00:00 for every 15 minutes time frame
01-09-2024 03:00 : 96.0 mg/dL
01-09-2024 03:15 : 109.0 mg/dL
01-09-2024 03:30 : 101.0 mg/dL
01-09-2024 03:45 : 105.0 mg/dL
01-09-2024 04:00 : 92.0 mg/dL
01-09-2024 04:15 : 99.0 mg/dL
01-09-2024 04:30 : 90.0 mg/dL
01-09-2024 04:45 : 106.0 mg/dL
01-09-2024 05:00 : 88.0 mg/dL
01-09-2024 05:15 : 75.0 mg/dL
01-09-2024 05:30 : 84.0 mg/dL
01-09-2024 05:45 : 92.0 mg/dL
01-09-2024 06:00 : 98.0 mg/dL
01-09-2024 06:15 : 106.0 mg/dL
01-09-2024 06:30 : 114.0 mg/dL
01-09-2024 06:45 : 117.0 mg/dL
01-09-2024 07:00 : 127.0 mg/dL
01-09-2024 07:15 : 195.0 mg/dL
01-09-2024 07:30 : 239.0 mg/dL
01-09-2024 07:45 : 245.0 mg/dL
01-09-2024 08:00 : 211.0 mg/dL
01-09-2024 08:15 : 195.0 mg/dL
01-09-2024 08:30 : 174.0 mg/dL
01-09-2024 08:45 : 158.0 mg/dL
01-09-2024 09:00 : 88.2 mg/dL
01-09-2024 09:15 : 89.1 mg/dL
01-09-2024 09:30 : 108.8 mg/dL
01-09-2024 09:45 : 111.7 mg/dL
01-09-2024 10:00 : 121.0 mg/dL
01-09-2024 10:15 : 135.7 mg/dL
01-09-2024 10:30 : 135.7 mg/dL
01-09-2024 10:45 : 169.2 mg/dL
01-09-2024 11:00 : 196.0 mg/dL

"""

In [46]:
import json
from datetime import datetime,timedelta

with open(r"C:\Users\Vishwas\Documents\food_fabricated.json") as f:
    food_data = json.load(f)
    # food_data = food_data["select * from tbl_customer_iot_data tcid where customer_id='261cb253-f11a-461c-9dc0-b608549c4454'"]

with open(r"C:\Users\Vishwas\Documents\cgm_fabricated.json") as f:
    cgm_data = json.load(f)

In [167]:
reference_time_str = "01-09-2024 18:00"
reference_time = datetime.strptime(reference_time_str, "%d-%m-%Y %H:%M")
start_time = reference_time - timedelta(hours=8)

# Filter readings within the last 2 hours before the reference time
cgm8 = ""
filtered_glucose_readings = [
    {
        "glucose_reading": str(reading["glucose_reading"]) + " "+"mg/dL",
        "reading_date_time": reading["reading_date_time"],
        # "meals_data": fetch_meals(reading["reading_date_time"]),
        # "Activity": fetch_steps(reading['reading_date_time'])
    }
    for reading in cgm_data
    if start_time <= datetime.strptime(reading["reading_date_time"], "%d-%m-%Y %H:%M") <= reference_time
]
cgm8 += f"CGM readings starting from : {start_time} for every 15 minutes timeframe\n"
for i,entry in enumerate(filtered_glucose_readings,1):
        cgm8 += f"{entry['glucose_reading']} at {entry['reading_date_time']}\n"

In [168]:
print(cgm8)

CGM readings starting from : 2024-09-01 10:00:00 for every 15 minutes timeframe
121.0 mg/dL at 01-09-2024 10:00
135.7 mg/dL at 01-09-2024 10:15
135.7 mg/dL at 01-09-2024 10:30
169.2 mg/dL at 01-09-2024 10:45
196.0 mg/dL at 01-09-2024 11:00
198.9 mg/dL at 01-09-2024 11:15
187.2 mg/dL at 01-09-2024 11:30
168.2 mg/dL at 01-09-2024 11:45
162.6 mg/dL at 01-09-2024 12:00
154.8 mg/dL at 01-09-2024 12:15
77.9 mg/dL at 01-09-2024 12:30
126.1 mg/dL at 01-09-2024 12:45
126.1 mg/dL at 01-09-2024 13:00
185.6 mg/dL at 01-09-2024 13:15
236.7 mg/dL at 01-09-2024 13:30
251.4 mg/dL at 01-09-2024 13:45
221.2 mg/dL at 01-09-2024 14:00
202.2 mg/dL at 01-09-2024 14:15
172.4 mg/dL at 01-09-2024 14:30
179.3 mg/dL at 01-09-2024 14:45
75.0 mg/dL at 01-09-2024 15:00
95.5 mg/dL at 01-09-2024 15:15
118.5 mg/dL at 01-09-2024 15:30
125.8 mg/dL at 01-09-2024 15:45
125.8 mg/dL at 01-09-2024 16:00
152.8 mg/dL at 01-09-2024 16:15
170.7 mg/dL at 01-09-2024 16:30
178.7 mg/dL at 01-09-2024 16:45
165.4 mg/dL at 01-09-2024 1

In [169]:
filtered_step_readings = [
      reading
    for reading in food_data
    if start_time <= datetime.strptime(reading["mealTime"], "%d-%m-%Y %H:%M") <= reference_time
]
food8= f"food data along with the respective timeframes\n"
for i, entry in enumerate(filtered_step_readings, 1):
    # prompt += f"{i}. Time: {entry['reading_date_time']}\n"
    # prompt += f"   Glucose: {entry['glucose_reading']}\n"

    food8+=f"{entry['foodName']} (Protein- {entry['nutrition']['Protein']}, Carbs- {entry['nutrition']['Carbohydrate']}, Energy- {entry['nutrition']['Energy']}, Fiber- {entry['nutrition']['Fiber']}) consumed at {entry['mealTime']}\n"



In [170]:
print(food8)

food data along with the respective timeframes
Tea (Protein- 3.58 g, Carbs- 12.64 g, Energy- 101.45 Kcal, Fiber- 0.0 g) consumed at 01-09-2024 10:30
Marie light biscuit (Protein- 0.94 g, Carbs- 7.4 g, Energy- 45.7 Kcal, Fiber- 0.0 g) consumed at 01-09-2024 10:30
White Rice  (Protein- 5.07 g, Carbs- 49.98 g, Energy- 227.63 Kcal, Fiber- 1.79 g) consumed at 01-09-2024 13:30
Drumstick sambar (Protein- 5.18 g, Carbs- 19.7 g, Energy- 140.82 Kcal, Fiber- 4.13 g) consumed at 01-09-2024 13:30
Chow chow poriyal (Protein- 1.29 g, Carbs- 5.03 g, Energy- 29.84 Kcal, Fiber- 2.02 g) consumed at 01-09-2024 13:30
Filter Coffee (Protein- 4.26 g, Carbs- 12.91 g, Energy- 118.06 Kcal, Fiber- 0.0 g) consumed at 01-09-2024 16:00
Masala Vada (Protein- 2.48 g, Carbs- 6.16 g, Energy- 81.41 Kcal, Fiber- 2.09 g) consumed at 01-09-2024 18:00



In [171]:
import pandas as pd
df = pd.read_csv(r"c:\Users\Vishwas\Documents\fabricated_step.csv")
# p=""
steps8 = f"Steps data along with the respective timeframes\n"
for index, row in df.iterrows():  
      try:    
            
            ctime = datetime.strptime(row['Date Time'], "%m-%d-%Y %H:%M")
            # print(ctime)
            if start_time <= ctime <= reference_time:
                  date_obj = datetime.strptime(row['Date Time'], "%d-%m-%Y %H:%M")

# Convert to desired format (MM-DD-YYYY)
                  new_date = date_obj.strftime("%m-%d-%Y %H:%M")
                  steps8+=f"{str(row['Steps'])} Steps at {new_date}\n"
      except:
            pass

In [172]:
print(steps8)

Steps data along with the respective timeframes
5.0 Steps at 01-09-2024 10:00
50.0 Steps at 01-09-2024 11:00
55.0 Steps at 01-09-2024 12:00
5.0 Steps at 01-09-2024 13:00
5.0 Steps at 01-09-2024 14:00
5.0 Steps at 01-09-2024 15:00
226.0 Steps at 01-09-2024 16:00
15.0 Steps at 01-09-2024 17:00
20.0 Steps at 01-09-2024 18:00



In [173]:
# from openai import OpenAI
# from openai import AzureOpenAI

# import os
# # from dotenv import load_dotenv
# # load_dotenv()

# client = OpenAI( api_key=os.environ.get("OPENAI_API_KEY"))


# completion = client.chat.completions.create(
#     # model="gpt-4o",
#     # model="gpt-4o-mini",
#     # model="gpt-4.1",
#     # model="gpt-4.1-mini",
#     model="gpt-4.1-nano",
#     # model="o3-mini",
#     messages=[
#         {"role": "system",
#          "content":
#          """
#               You are a smart glucose monitoring assistant. Analyze continuous glucose monitor (CGM) data recorded every 15 minutes along with corresponding meal intake and activity levels (steps walked) and sleep duration. Review the entire timeline of data, but generate a user-friendly notification only if:
#                 - Glucose > 180 mg/dL → classify as a "spike" and identify the most probable cause and suggest the reason for the same
#                 - Glucose < 80 mg/dL → classify as a "drop" and identify the most probable cause and suggest the reason for the same

#               Input format:

#               cgm: Multiline string with CGM readings starting from a specific timestamp in dd-mm-yyyy HH:MM format, with readings every 15 minutes.
#               food: Multiline string with food intake information, each entry containing time(dd-mm-yyyy HH:MM), foodName, and nutritional details (Protein, Carbs, Energy, Fiber).
#               steps: Multiline string with steps walked and associated timestamps(dd-mm-yyyy HH:MM).
#               sleep: single line string with hrs and mins (ex: 5hrs 30mins)
#               gender: male or female

#               Evaluate contributing factors when generating glucose alerts. Consider the following:
#                 - food: Strictly check for recent food intake (within 1.5-2 hours of spike), especially high glycemic foods that may cause spikes.
#                 - Skipped meals or fasting: May lead to drops or instability in glucose.
#                 - Poor sleep (<6 hours): May explain elevated glucose or fatigue and doesn't justify glucose drop.
#                 - Morning effects (dawn phenomenon): In early hours, natural hormone release (cortisol, growth hormone) may elevate glucose before breakfast.
#                 - Activity: Low or no post-meal movement may contribute to postprandial spikes due to reduced glucose uptake by muscles.
#                 - gender: Identify the potential cause and generate the response considering the gender of a person.
#                 - age: Identify the potential cause and generate the response considering the gender of a person.
#               Generate a concise alert (max 30 words) that includes the timestamp, glucose value, the most probable reason, and a brief suggestion to improve. Ensure the message is clear and mobile-friendly.


#             Example Notification Output:
#             1.   Glucose spike (182 mg/dL) at 06:45, likely due to early morning hormone changes. Try avoiding late-night snacks and see how your morning readings go.
#             2.   Glucose spike (192 mg/dL), due to white rice at 13:00. Consider a light walk post-meal.
#             3.   Glucose drop (72 mg/dL) after skipping lunch. Your energy dipped. Try not to delay meals—have a small snack like a banana or a handful of roasted chana.
#             4.   Glucose (105 mg/dL) at 08:15. Slightly elevated, possibly due to stress from poor sleep. A brisk 10-minute walk can help balance energy and improve glucose control.

#             **note: make sure the response must be same as one of the example notifications and within 30 words maximum.**
                
#         """
#          },
#         {
#             "role": "user",
#             "content": f"""
#               cgm:{cgm8},food:{food8},steps:{steps8},sleep:7hrs 30mins
#               Based on all the above observations help me suggest the reason for the glucose levels at {reference_time_str}
#             """
            
#         },
#     ],
# )

# print(completion.choices[0].message.content)

In [181]:
from openai import OpenAI
from openai import AzureOpenAI

import os
# from dotenv import load_dotenv
# load_dotenv()

client = OpenAI( api_key=os.environ.get("OPENAI_API_KEY"))


completion = client.chat.completions.create(
    # model="gpt-4o",
    # model="gpt-4o-mini",
    # model="gpt-4.1",
    # model="gpt-4.1-mini",
    # model="gpt-4.1-nano",
    model="o3-mini",
    messages=[
        {"role": "system",
         "content":
         """
              You are a smart glucose monitoring assistant. Analyze Continuous Glucose Monitor (CGM) data recorded every 15 minutes, along with food intake, activity levels (steps walked), sleep duration, gender, and age. Review the full timeline and generate a user-friendly notification only if:
                - Glucose > 180 mg/dL → classify as a "spike"
                - Glucose < 80 mg/dL → classify as a "drop"

             Input Format:
                  ""
                        cgm: Multiline string. Each line contains a CGM reading (mg/dL) and timestamp in dd-mm-yyyy HH:MM format, spaced 15 minutes apart.
                        food: Multiline string. Each line includes: timestamp, foodName, and nutritional values (Protein, Carbs, Energy, Fiber).
                        steps: Multiline string. Each line contains a timestamp and step count.
                        sleep: Single-line string in "5hrs 30mins" format.
                        gender: "male" or "female"
                        age: Integer (e.g., 35)
                  ""
              Evaluation Logic (for notifications)
                  When generating alerts, analyze contributing factors like:
                  Food intake within 1.5-2 hours before the spike, especially high-glycemic or high-carb foods.
                  Skipped or delayed meals may contribute to glucose drops or instability.
                  Poor sleep (<6 hours) may cause elevated glucose or fatigue but not drops.
                  Dawn phenomenon: Early morning spikes may occur naturally due to hormonal changes.
                  Low post-meal activity can cause spikes due to reduced glucose uptake.
                  Age and gender: Tailor the response subtly (physiological tendencies), without explicitly mentioning them in the alert.

             Generate a concise alert (max 30 words) that includes the timestamp, glucose value, the most probable reason, and a brief suggestion to improve. Ensure the message is clear and mobile-friendly.

            
            Example Notifications (max 30 words, implicitly tailored):
               1.  Glucose spike (182 mg/dL) at 06:45, likely due to early morning hormone changes. Try avoiding late-night snacks and see how your morning readings go.
               2.   Glucose spike (192 mg/dL) at 13:00, due to white rice. Consider a light walk post-meal to help with glucose absorption.
               3.   Glucose drop (72 mg/dL) at 15:30, after skipping lunch. Your energy dipped. Try not to delay meals—have a small snack like a banana or a handful of roasted chana.
               4.   Glucose drop (78 mg/dL) at 10:15, following an active morning and no breakfast. Add a small protein-carb combo early in the day to maintain balance.
               5.   Glucose spike (198 mg/dL) at 19:00, due to sugary tea and low movement. Include a short post-meal walk or reduce added sugars in the evening.
               6.   Glucose spike (184 mg/dL) at 07:15, possibly a morning spike. Try a protein-rich breakfast and get sunlight exposure to reset your rhythm.
               7.   Glucose drop (76 mg/dL) at 12:45, likely from delaying meals after a long walk. Have a snack with protein and carbs earlier in your routine.
            
            **note - make sure the response output size must be similar as one of the example notifications and within 30 words maximum.**
                
        """
         },
        {
            "role": "user",
            "content": f"""
              cgm:{cgm8},food:{food8},steps:{steps8},sleep:7hrs 30mins,age:54,gender:male
              Based on all the above observations help me suggest the reason for the glucose levels at {reference_time_str}
            """
            
        },
    ],
)

print(completion.choices[0].message.content)

Glucose drop (68.5 mg/dL) at 18:00, likely from reactive hypoglycemia after an earlier high-carb meal. Consider a balanced protein-carb dinner to help stabilize levels.


In [160]:
reference_time_str

'01-09-2024 20:45'